In [1]:
# bench marking
import timeit
import pandas as pd
import numpy as np
import classification_tree
import SQL
import psycopg2
import cProfile
import pstats
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier

In [2]:
impurity_function = classification_tree.bayers_error

conn = psycopg2.connect(host="sculptor.stat.cmu.edu", database="zhiyanh",
            user="zhiyanh", password="ahY0ieMou")
cur = conn.cursor()

data = load_breast_cancer()

data_d = pd.DataFrame(data.data[:,[0,1,2,3,4]])
data_d = data_d.head(50)
data_t = pd.DataFrame(data.target)
data_t = data_t.head(50)
data_1 = pd.concat((data_d,data_t), axis = 1)

feature_names = ["col1","col2","col3","col4","col5","Y"]
data_1.columns = np.asarray(feature_names)
cols = ",".join([str(i) for i in data_1.columns.tolist()])

cur.execute("CREATE TEMP TABLE benchmark_SQL (col1 numeric, col2 numeric, col3 numeric, col4 numeric,col5 numeric, Y integer) ON COMMIT DELETE ROWS;")
cur.execute("BEGIN TRANSACTION;")
for i,row in data_1.iterrows():
    sql = "INSERT INTO benchmark_SQL (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cur.execute(sql, tuple(row))


In [5]:
# Benchmarking for build and prune tree

# for pd dataframe
%time clt_pd = classification_tree.Tree(data_1, 'Y', classification_tree.bayers_error)
# Time for building and puring a tree with pd database is 30.7 s


Wall time: 30.7 s


In [12]:
# for sql database
%time clt_sql = SQL.SQL_Tree("benchmark_SQL", ["col1","col2","col3","col4","col5"], "Y","gini_index", cur, 100,0.8)
# Time for building and puring a tree with SQL database is 16 s

Wall time: 16 s


In [17]:
# Profiling for pd trees
cProfile.run("classification_tree.Tree(data_1, 'Y', classification_tree.bayers_error)","result_pd_tree")
p = pstats.Stats('result_pd_tree')
p.strip_dirs().sort_stats('cumulative').print_stats(30)

Sun Dec  8 15:15:19 2019    result

         81110868 function calls (79729604 primitive calls) in 53.864 seconds

   Ordered by: cumulative time
   List reduced from 346 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   53.864   53.864 {built-in method builtins.exec}
        1    0.000    0.000   53.864   53.864 <string>:1(<module>)
        1    0.000    0.000   53.864   53.864 classification_tree.py:437(__init__)
   320/36    0.008    0.000   53.671    1.491 classification_tree.py:16(__init__)
      179    0.302    0.002   53.261    0.298 classification_tree.py:160(find_best_splitting_point)
        1    0.000    0.000   52.372   52.372 classification_tree.py:466(pruning)
        1    0.006    0.006   52.372   52.372 classification_tree.py:484(cross_validation)
    25055    0.213    0.000   41.672    0.002 classification_tree.py:100(get_data)
   203489    0.814    0.000   29.089    0.000 frame.py:2893(

In [18]:
# Profileing for sql trees
cProfile.run('SQL.SQL_Tree("benchmark_SQL", ["col1","col2","col3","col4","col5"], "Y","gini_index", cur, 100,0.8)', "result_sql_tree")
p = pstats.Stats('result_sql_tree')
p.strip_dirs().sort_stats('cumulative').print_stats(30)

Sun Dec  8 15:19:07 2019    result_sql_tree

         23224 function calls (23213 primitive calls) in 14.547 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   14.547   14.547 {built-in method builtins.exec}
        1    0.000    0.000   14.547   14.547 <string>:1(<module>)
        1    0.000    0.000   14.547   14.547 SQL.py:344(__init__)
      9/1    0.001    0.000   14.547   14.547 SQL.py:11(__init__)
        4    0.088    0.022   14.418    3.605 SQL.py:108(find_best_splitting_point)
     3158   14.353    0.005   14.408    0.005 {method 'execute' of 'psycopg2.extensions.cursor' objects}
     6321    0.022    0.000    0.055    0.000 utf_8.py:15(decode)
     6321    0.033    0.000    0.033    0.000 {built-in method _codecs.utf_8_decode}
      996    0.026    0.000    0.026    0.000 {built-in method builtins.min}
     3138    0.017    0.000    0.017    0.000 {method 'fetchone' of 'psycopg2.exten

In [7]:
cProfile.run('classification_tree.random_forest(data_1, 2 , 50, "Y", classification_tree.gini)','result_pd_rf')
p_1 = pstats.Stats('result_pd_rf')
p_1.strip_dirs().sort_stats('cumulative').print_stats()

Sun Dec  8 15:29:02 2019    result_pd_rf

         29105657 function calls (28613856 primitive calls) in 19.740 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   19.740   19.740 {built-in method builtins.exec}
        1    0.000    0.000   19.740   19.740 <string>:1(<module>)
        1    0.001    0.001   19.740   19.740 classification_tree.py:645(__init__)
       50    0.001    0.000   19.711    0.394 classification_tree.py:551(__init__)
   298/50    0.006    0.000   19.710    0.394 classification_tree.py:16(__init__)
     8886    0.079    0.000   15.098    0.002 classification_tree.py:100(get_data)
       44    0.071    0.002   12.250    0.278 classification_tree.py:576(find_best_splitting_point)
    73442    0.300    0.000   10.516    0.000 frame.py:2893(__getitem__)
    17772    0.082    0.000    8.728    0.000 frame.py:2952(_getitem_bool_array)
    17910    0.145    0.000    7.632    0.000

In [10]:
clf = RandomForestClassifier(n_estimators = 50,max_features = 2)
rv = data_1[["Y"]].values.ravel()

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features=2, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [11]:
cProfile.run('clf.fit(data_1[["col1","col2","col3","col4","col5"]], rv)','result_sklearn_rf')
p = pstats.Stats('result_sklearn_rf')
p.strip_dirs().sort_stats('cumulative').print_stats()

Sun Dec  8 15:36:03 2019    result_sklearn_rf

         77445 function calls (76744 primitive calls) in 0.052 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.052    0.052 {built-in method builtins.exec}
        1    0.000    0.000    0.052    0.052 <string>:1(<module>)
        1    0.000    0.000    0.050    0.050 forest.py:217(fit)
        1    0.000    0.000    0.031    0.031 forest.py:315(<listcomp>)
       50    0.000    0.000    0.031    0.001 base.py:120(_make_estimator)
      250    0.002    0.000    0.026    0.000 base.py:177(get_params)
      250    0.002    0.000    0.023    0.000 base.py:150(_get_param_names)
      250    0.000    0.000    0.018    0.000 inspect.py:3081(signature)
      250    0.000    0.000    0.018    0.000 inspect.py:2829(from_callable)
        1    0.000    0.000    0.018    0.018 parallel.py:866(__call__)
      250    0.001    0.000    0.018    0.000 inspec

        3    0.000    0.000    0.000    0.000 series.py:382(_set_subtyp)
        1    0.000    0.000    0.000    0.000 {pandas._libs.internals.get_blkno_placements}
        7    0.000    0.000    0.000    0.000 common.py:122(classes_and_not_datetimelike)
        2    0.000    0.000    0.000    0.000 managers.py:308(<genexpr>)
        1    0.000    0.000    0.000    0.000 {method 'take' of 'numpy.ndarray' objects}
        8    0.000    0.000    0.000    0.000 {built-in method _thread.get_ident}
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap_external>:36(_relax_case)
        1    0.000    0.000    0.000    0.000 generic.py:5036(__finalize__)
        2    0.000    0.000    0.000    0.000 {method 'fill' of 'numpy.ndarray' objects}
        3    0.000    0.000    0.000    0.000 {built-in method __new__ of type object at 0x00007FFF6EF56BA0}
        5    0.000    0.000    0.000    0.000 {method 'join' of 'str' objects}
        2    0.000    0.000    0.000    0.000 {